# Splash Test 💧💧

<div class="alert alert-block alert-success">

## Part 2: Generating the Initialization `.yaml` File

Here we will walk through the generation of the `.yaml` file for the splash test on a cubed-sphere. Since this is the first time we are departing from a cartesian geometry, as well as exploring a new equation-of-state, we will go step-by-step through the file (we recommend having the `shallow_splash.yaml` file up while going through this).

<img src="../../images/cubed_sphere.png">

From [Chen & Li 2024](https://ui.adsabs.harvard.edu/abs/2024ApJ...966..123C/abstract)

<div class="alert alert-info">

### Define Simulation Domain Geometry ($\texttt{geometry}$)

The first dictionary entry in the `.yaml` file, $\texttt{geometry}$, defines the geometry of the simulation domain. 

Here, we will utilize the `gnomonic-equiangle` geometry, which is just a fancy name for the coordinate system that allows for the transformation between a cube and sphere. 

The cubed sphere is generated from 6 panels of an unfolded sphere (see figure above). We define our bounds for a **single** panel, which then propagates to each of the 6 panels. 

The first dimension of this geometry (x1) is the exact same as cartesian: height. To be more specific `x1 defines the altitude range (in meters) from the center of the sphere that the simulation will be run in.`

x2 and x3 are then defined as angular coordinates. `x2_pi and x3_pi will always range from -pi/4 to +pi/4 for the cubed sphere geometry where an entire planet is being simulated.`

If one instead wishes to model just a portion of a planet (like a polar cap, or tropical channel) while taking a spherical geometry into account, these values can be adjusted. 

*One might naively assume these numbers should be pi/2 instead of pi/4. However, each panel of the cube is defined from its origin. Therefore, the angular span of each panel pi/2.*

The next thing we do is decide the resolution (i.e., number of cells) that belongs to each panel. **Since this is a shallow water model, the number of cells in the vertical direction is 1.**

The angular resolution can be backed out by considering that four panels span the equatorial region of the sphere. Therefore, if nx2 and nx3 are equal, the angular resolution is equal to 

$\mathrm{Angular \; Resolution} = \frac{360^{\circ}}{4*n_{x2}}$

Here, we will set `nx2 = nx3 = 48`, with an average angular resolution of 1.875 x 1.875 degrees. 

</div>

In [1]:
# Here we define spatial dimensions of each panel 

# Initialize a cubed-sphere geometry 
geometry_type = 'gnomonic-equiangle'

# Spatial Dimensions ('bounds')

# x1 = Height (often denoted with 'z')
# Here, we define the simulation domain at the surface of Earth 
# 1 meter high 
x1min = 6371000
x1max = 6371001 # in meters

# x2 = Angular extent left/right of a single panel
x2min_pi = -0.25
x2max_pi = 0.25 # radians

# x3 = Angular extent up/down of a single panel 
x3min_pi = -0.25
x3max_pi = 0.25 # radiants

# Spatial Resolutions ('cells')

# nx1 = Number of Cells in Height Dimension
# In shallow water, we only have one layer in the height dimension
nx1 = 1 # 1 cell

# nx2 = Number of Cells in Left/Right Angular Definition of a Single Panel
nx2 = 48 # Results in an angular resolution of 1.875 degrees

# nx3 = Number of Cells in Up/Down Angular Definition of a Single Panel
nx3 = 48 # Results in an angular resolution of 1.875 degrees 

# nghost = Number of ghost zones
# The ghost zones are additional cells that pad the interior simulation domain 
# A good default number here is 3
nghost = 3

Now we actually create the dictionary entry for geometry from the variables above, so we can generate the `.yaml` file from a dictionary at the end of this section.

In [2]:
geometry_dict = {'type': geometry_type,
                 'bounds': {'x1min': x1min, 'x1max': x1max,
                            'x2min_pi': x2min_pi, 'x2max_pi': x2max_pi, 
                            'x3min_pi': x3min_pi, 'x3max_pi': x3max_pi},
                 'cells':  {'nx1': nx1, 'nx2': nx2, 'nx3': nx3, 'nghost': nghost}}

<div class="alert alert-info">

### Define Dynamics ($\texttt{dynamics}$)

The second dictionary entry, $\texttt{dynamics}$, defines the dynamical properties of the simulation. 

In this context, $\texttt{dynamics}$ means properties like the equation of state of the atmosphere, as well as defining the algorithms that are utilized to solve the hydrodynamics of the simulation. 

The $\texttt{dynamics}$ dictionary entry is composed of multiple sub-dictionaries, defined below.

</div>

#### Dynamics sub-dictionary: Equation-of-State Properties 

Here we will define the equation of state to be `shallow-water`. Here, there are no additional parameters we have to define. 

In [3]:
# Equation of State (EOS) Properties

eos_type = 'shallow-water'

# Make Sub-Dictionary 
equation_of_state_dict = {'type': eos_type}

#### Dynamics sub-dictionaries: Vertical Projection, Reconstruct, and Riemann Solver Properties 

The following sub-dictionaries of $\texttt{dynamics}$ (which determine different algorithms that are used in the simulation, and therefore numerical stability) can be taken as-is, though we will provide short introductions to the following: 

Unlike Straka and Robert, $\texttt{vertical-projection}$ is not needed here since we only have one cell in the vertical direction.

The $\texttt{reconstruct}$ properties define the algorithm to compute hydrodynamic variables at cell boundaries in the simulation from properties like cell averages. Here we use the [WENO5](https://en.wikipedia.org/wiki/WENO_methods) (fifth-order accurate weighted essentially non-oscillation) algorithm, which is generally stable for a variety of applications.

Finally we have the $\texttt{riemann-solver}$, which computers fluxes given hydrodynamic variables at cell boundaries. For the shallow water, we utilize the riemann solver developed by Dr. Phillip Roe (University of Michigan) specifically for shallow water dynamical simulations. 

Shallow water has two modes: xy and yz. `Need more details here`


In [4]:
# Reconstruct Properties 

# Uses the weno5 algorithm 

# Vertical Reconstruction 
vertical_reconstruct_type = 'weno5'
vertical_reconstruct_scale = False
vertical_reconstruct_shock = False

# Horizontal Reconstruction 
horizontal_reconstruct_type = 'weno5'
horizontal_reconstruct_scale = False
horizontal_reconstruct_shock = False

# Make Sub-Dictionary 
reconstruct_dict = {'vertical'  : {'type' : vertical_reconstruct_type,
                                   'scale': vertical_reconstruct_scale,
                                   'shock': vertical_reconstruct_shock},
                    'horizontal': {'type' : horizontal_reconstruct_type,
                                   'scale': horizontal_reconstruct_scale,
                                   'shock': horizontal_reconstruct_shock}}

In [5]:
# Riemann Solver 

# Shallow water solver developed by Dr. Phillip Roe
riemann_type = 'shallow-roe'
dir = 'yz'

# Make Sub-Dictionary 
riemann_solver_dict = {'type': riemann_type,
                       'dir': dir}

Now we create the dictionary object for the future .yaml file

In [6]:
# Combine the dictionaries above into a larger dynamics dictionary 

dynamics_dict = {'equation-of-state': equation_of_state_dict,
                 'reconstruct': reconstruct_dict,
                 'riemann-solver': riemann_solver_dict}

<div class="alert alert-info">

### Define Boundary Conditions ($\texttt{boundary-condition}$)

The third dictionary entry, $\texttt{boundary-condition}$, defines how waves interact with the boundaries of the simulation domain. 

Here, we define `reflecting` (solid wall boundary that waves cannot cross) for the x1 direction (a requirement of the single-layer present in the shallow water). 

For x2 and x3, we utilize a `custom` boundary condition. `need more details here`

</div>

In [7]:
# External Properties 

# x1 (height) inner and outer properties 
x1_inner = 'reflecting' # Bottom Wall
x1_outer = 'reflecting' # Top Wall

# x2 = Angular extent left/right of a single panel
x2_inner = 'custom' # Left Wall
x2_outer = 'custom' # Right Wall

# x3 = Angular extent up/down of a single panel
x3_inner = 'custom' # Towards us Wall
x3_outer = 'custom' # Away from us Wall


Now we create the dictionary object for the future .yaml file

In [8]:
boundary_condition_dict = {'external': {'x1-inner': x1_inner,
                                         'x1-outer': x1_outer,
                                         'x2-inner': x2_inner,
                                         'x2-outer': x2_outer,
                                         'x3-inner': x3_inner,
                                         'x3-outer': x3_outer}}

<div class="alert alert-info">

### Define Integration Properties ($\texttt{integration}$)

The fourth dictionary entry, $\texttt{integration}$, determines how time-stepping is solved and handled in the simulation. In particular, it also sets how long a simulation is run for. 

$\texttt{integration-type}$ is a strong-stability preserving third-order Runge-Kutta method ('rk3') (for mathematical details, see [link](https://gkeyll.readthedocs.io/en/latest/dev/ssp-rk.html)). This is the default integration type, and there are not too many reasons to change this. 

$\texttt{cfl}$ is the [Courant-Friedrich-Lewy](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition) number which determines the stability of the time-stepping scheme. It is the the ratio of how far a wave/particle can travel in a single time step vs the local cell size. You want this number to be less than 1, where a cfl larger than 1 means information travels more than one cell in a time step, which is not stable.

We do not need to define an $\texttt{implicit-scheme}$, since we only have one cell in the vertical direction. 

</div>

In [9]:
# Integration properties 

# Integration type is a third-order Runge-Kutta method
integration_type = 'rk3'

# cfl = stability condition (Courant-Friedrich-Lewy number)
cfl = 0.9

Here we define properties that determine how long a simulation runs and how often diagnostic information is printed out. 

There are two important distinctions to be made: 

1. $\texttt{cycles}$: A cycle is an integration step, where the simulation time that has elapsed (*dt*) is determined in part by the cfl number above. 

2. $\texttt{time}$: Time here is the number of seconds that have elapsed in the simulation, not amount of time that has elapsed in real life. 

In [10]:
# nlim 
# Number of cycles to run model 
# -1 = run until time limit
nlim = -1 

# How long we want the simulation to run before it terminates
# Here, we set it at 8.64e5 seconds, or, 10 days
tlim = 8.64e5

# ncycle_out 
# frequency of printing out diagnostic information to terminal 
# cycle = integration step
ncycle_out = 10

Now we create the dictionary object for the future .yaml file

In [11]:
integration_dict = {'type': integration_type,
                    'cfl': cfl,
                    'nlim': nlim,
                    'tlim': tlim,
                    'ncycle_out':10}

<div class="alert alert-info">

### Define Forcing ($\texttt{forcing}$ )

For our shallow water model, we do not need to explicitly define gravity since it is included in the geopotential. 

</div>

<div class="alert alert-info">

### Define Outputs ($\texttt{outputs}$)

The final dictionary entry, $\texttt{outputs}$, determines what sorts of files are produced by the simulation. This portion is extremely important to define carefully, as it determines how often outputs are produced and what sort of variables are saved for future analysis. 

Here, we just output the primitive variables (no user output variables) that output, and opt to not define a cadence for .restart files (just the output .netcdf files)

</div>

In [12]:
# Output properties 

# We define how often we generate .nc files that track specific variables

# We first define which variables we want to be track 
# prim = primitive variables
variables_track = ['prim']

# Then we define how often we want the files to be generated 
# Here, its every 1e4 simulations seconds (~2.75 hours)
dt_netcdf = 1.E4

Now we create the dictionary object for the future .yaml file

In [13]:
outputs_dict = {'type': 'netcdf', 'variables' : variables_track, 'dt': dt_netcdf}

<div class="alert alert-info">

### Define Multi-Core Processing ($\texttt{distribute}$ )

As we discovered in Robert, our simulations have become a bit too advanced to run in notebook. Therefore, we are required to use multiple cores in a terminal command. 

While we will not go into a full description of the `distribute` dictionary below, we will note that 

`For cubed sphere, each of the six panels of the sphere is assigned a single core. So the code below allows the code to be run on six cores. In general, cubed-spheres always have to be run on cores that are a multiple of 6`.

`Figure out how to make it 12`

</div>

In [15]:
backend = 'gloo'

# Here, we use the cubed-sphere layout instead of cartesian 
layout = 'cubed-sphere'

# Note that the above allows for a default 6 cores to be utilized
nb2 = 1
nb3 = 1

verbose = False 

distribute_dict = {'backend': backend, 
                   'layout' : layout, 
                   'nb2': nb2,
                   'nb3': nb3,
                   'verbose': verbose}

<div class="alert alert-info">

### Create the input `.yaml` file 

Here, we generate the `.yaml` file from the dictionaries we defined above. 

<div>

In [16]:
import yaml

full_dictionary = {'geometry': geometry_dict,
                   'dynamics': dynamics_dict,
                   'boundary-condition': boundary_condition_dict,
                   'integration': integration_dict,
                   'outputs': outputs_dict,
                   'distribute': distribute_dict}

file_path = "shallow_splash_made_in_notebook.yaml"
with open(file_path, "w") as file_handler:
    yaml.dump(full_dictionary, file_handler, sort_keys=False)

<div class="alert alert-block alert-success">

Now we have our `.yaml` file with all the options needed to execute the shallow water splash test.

On we go to **Part 3**, where we discuss how to set up generate an execution script that is used to run a cubed-sphere $\texttt{PADDLE}$ simulation.